In [1]:
if "running_all" not in globals():
    from ipywidgets import widgets
    matrices = ["bcsstk18", "cbuckle", "ct20stif", "raefsky4", "vanbody"]
    b = widgets.Button(description="Run over all matrices", button_style="success")
    output = widgets.Output()

    display(b, output)

    def run_over_all_matrices(button):
        global running_all
        global matrix
        running_all = True
        with output:
            for matrix in matrices:
                print(f"Running {matrix}...")
                %run ./model_evaluation.ipynb # will output at this cell rather than later
            print("Finished!")
    b.on_click(run_over_all_matrices)

Button(button_style='success', description='Run over all matrices', style=ButtonStyle())

Output()

In [ ]:
if "running_all" not in globals():
    matrix = "ct20stif"  # manually set to run over one matrix
matrix

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import load
from glob import glob
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, fbeta_score

In [ ]:
test_data_sizes = {
    "bcsstk18": 1000,
    "cbuckle": 1000,
    "ct20stif": 1000,
    "raefsky4": 1000,
    "vanbody": 100
}

df = pd.read_csv(f"data/{matrix}_{test_data_sizes[matrix]}.csv")
df.head()

In [ ]:
df.sort_values("slowdown", inplace=False, ascending=False).head()

In [ ]:
X_test = df[["error_iter", "pos_2norm"]].to_numpy()
y_test = df["slowdown"].to_numpy()

In [ ]:
p = 1 / 98
1 + (1 / p)

# 1 + (1 / x) = 99
# (1 / x) = 98
# x = 1 / 98

In [ ]:
models = [load(fn) for fn in glob(f"./models/{matrix}/*.pkl")]
model_names = [model.steps[-1][1].__class__.__name__ for model in models]
ps = list(np.arange(1 / 98, 1, 0.01))

reports = pd.DataFrame(index=ps, columns=model_names)
reports.head()

In [ ]:
for p in ps:
    for model, name in zip(models, model_names):
        preds = model.predict(X_test)
        y_pred_clas = preds > (1 + (1 / p))
        y_true_clas = y_test > (1 + (1 / p))
        reports.loc[p][name] = {
            "accuracy": accuracy_score(y_true_clas, y_pred_clas),
            "f1_score": f1_score(y_true_clas, y_pred_clas, zero_division=0),
            "precision": precision_score(y_true_clas, y_pred_clas, zero_division=0),
            "recall": recall_score(y_true_clas, y_pred_clas),
            "f1b_0.5_score": fbeta_score(y_true_clas, y_pred_clas, beta=0.5, zero_division=0),
            "f1b_2_score": fbeta_score(y_true_clas, y_pred_clas, beta=2, zero_division=0)
        }

In [ ]:
for name, series in reports.items():
    accuracies = [item["accuracy"] for item in series]
    best_acc_p = ps[np.argmax(accuracies)]
    f1_scores = [scores["f1_score"] for scores in series]
    best_f1_p = ps[np.argmax(f1_scores)]

    precisions = [scores["precision"] for scores in series]
    best_precision_p = ps[np.argmax(precisions)]
    recalls = [scores["recall"] for scores in series]
    best_recall_p = ps[np.argmax(recalls)]

    f1b_05s = [scores["f1b_0.5_score"] for scores in series]
    best_f1b_05_p = ps[np.argmax(f1b_05s)]
    f1b_2s = [scores["f1b_2_score"] for scores in series]
    best_f1b_2_p = ps[np.argmax(f1b_2s)]


    plt.plot(ps, accuracies, label=f"accuracy", c="r")
    plt.axvline(x=best_acc_p, linestyle='--', c="r")
    plt.annotate(f"{round(np.max(accuracies), 3)}", xy=(best_acc_p + 0.01, 0.5), c="r")

    plt.plot(ps, f1_scores, label=f"f1-score", c="g")
    plt.axvline(x=best_f1_p, linestyle='--', c="g")
    plt.annotate(f"{round(np.max(f1_scores), 3)}", xy=(best_f1_p + 0.01, 0.55), c="g")

    plt.plot(ps, precisions, label=f"precision", c="b")
    plt.axvline(x=best_precision_p, linestyle='--', c="b")
    plt.annotate(f"{round(np.max(precisions), 3)}", xy=(best_precision_p + 0.01, 0.6), c="b")

    plt.plot(ps, recalls, label=f"recall", c="y")
    plt.axvline(x=best_recall_p, linestyle='--', c="y")
    plt.annotate(f"{round(np.max(recalls), 3)}", xy=(best_recall_p + 0.01, 0.65), c="y")

    plt.plot(ps, f1b_05s, label=f"f1beta=0.5", c="magenta")
    plt.axvline(x=best_f1b_05_p, linestyle='--', c="magenta")
    plt.annotate(f"{round(np.max(f1b_05s), 3)}", xy=(best_f1b_05_p + 0.01, 0.7), c="magenta")
    
    plt.plot(ps, f1b_2s, label=f"f1beta=2", c="grey")
    plt.axvline(x=best_f1b_2_p, linestyle='--', c="grey")
    plt.annotate(f"{round(np.max(f1b_2s), 3)}", xy=(best_f1b_2_p + 0.01, 0.75), c="grey")

    plt.title(name)
    plt.xlabel("$p$")
    plt.ylabel("Classification Scores")
    plt.xlim(1/98, 1)
    plt.ylim(0, 1)
    plt.legend()
    os.makedirs(f"./figures/{matrix}", exist_ok=True)
    plt.savefig(f"./figures/{matrix}/{name}.png")
    plt.show()

In [ ]:
avg_metrics = pd.DataFrame()  # columns=["model", "metric", "value"]

avg_metrics["accuracy"] = reports.apply(lambda col: np.mean([scores["accuracy"] for scores in col]))
avg_metrics["f1_score"] = reports.apply(lambda col: np.mean([scores["f1_score"] for scores in col]))
avg_metrics["precision"] = reports.apply(lambda col: np.mean([scores["precision"] for scores in col]))
avg_metrics["recall"] = reports.apply(lambda col: np.mean([scores["recall"] for scores in col]))
avg_metrics["f1b_0.5_score"] = reports.apply(lambda col: np.mean([scores["f1b_0.5_score"] for scores in col]))
avg_metrics["f1b_2_score"] = reports.apply(lambda col: np.mean([scores["f1b_2_score"] for scores in col]))
avg_metrics = avg_metrics.reset_index(names=["model"])
avg_metrics = avg_metrics.melt(id_vars="model", var_name="metric")

avg_metrics

In [ ]:
best_models = avg_metrics.loc[avg_metrics.groupby('metric')['value'].idxmax()]
print(f'Best model for f1 is {best_models["model"][best_models["metric"] == "f1_score"].iloc[0]}')
print(f'Best model for f1b_0.5 is {best_models["model"][best_models["metric"] == "f1b_0.5_score"].iloc[0]}')
print(f'Best model for f1b_2 is {best_models["model"][best_models["metric"] == "f1b_2_score"].iloc[0]}')

best_models

In [ ]:
plt.gcf().set_size_inches(12, 6)
plt.gcf().set_dpi(100)
sns.barplot(avg_metrics, x="model", y="value", hue="metric", palette="hls")
plt.ylim(0, 1.3)
plt.title("Avg Metrics by Model")
plt.legend(ncols=2)
plt.savefig(f"./figures/{matrix}/model_avg_metrics.png")
plt.show()